In [1]:
import pandas as pd
import os
import sys
import glob
import time
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
from IPython.display import display, clear_output
from pyspark.sql import SQLContext

In [2]:
from pyspark.sql import SparkSession,functions, Window
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark.pandas as ps
from time import sleep

### Preparing .csv files. 
####  Spliting .csv file into a few ones

In [3]:
def ReadingCsvFile():
    Path = r"C:\My projects\Streaming_Data_Projects\SparkStructuredStreaming_Project1\Deutschland_Cities.csv"
    return pd.read_csv(Path)

data = ReadingCsvFile()
data 

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Berlin,52.5167,13.3833,Germany,DE,Berlin,primary,3644826.0,3644826.0
1,Hamburg,53.5500,10.0000,Germany,DE,Hamburg,admin,1841179.0,1841179.0
2,Munich,48.1372,11.5755,Germany,DE,Bavaria,admin,1471508.0,1471508.0
3,Cologne,50.9422,6.9578,Germany,DE,North Rhine-Westphalia,NaN,1085664.0,1085664.0
4,Frankfurt,50.1136,8.6797,Germany,DE,Hesse,minor,753056.0,753056.0
...,...,...,...,...,...,...,...,...,...
607,Rotenburg,53.0851,9.3879,Germany,DE,Lower Saxony,minor,NaN,NaN
608,Karlstadt,49.9603,9.7724,Germany,DE,Bavaria,minor,NaN,NaN
609,Neustadt,49.5656,8.8437,Germany,DE,Hesse,minor,NaN,NaN
610,Homberg,51.0299,9.4026,Germany,DE,Hesse,minor,NaN,NaN


In [4]:
k = 7
size = 100
 
for i in range(k):
    df = data[size*i:size*(i+1)]
    df.to_csv(f'Deutschland_Cities_{i+1}.csv', index=False)


df_1 = pd.read_csv("Deutschland_Cities_1.csv")
df_1.to_csv('C:\My projects\Streaming_Data_Projects\SparkStructuredStreaming_Project1\InputStreamingData\Deutschland_Cities_1.csv', 
            sep=',',index=False)

df_2 = pd.read_csv("Deutschland_Cities_2.csv")
df_2.to_csv('C:\My projects\Streaming_Data_Projects\SparkStructuredStreaming_Project1\InputStreamingData\Deutschland_Cities_2.csv', 
            sep=',',index=False)

df_3 = pd.read_csv("Deutschland_Cities_3.csv")
df_3.to_csv('C:\My projects\Streaming_Data_Projects\SparkStructuredStreaming_Project1\InputStreamingData\Deutschland_Cities_3.csv', 
            sep=',',index=False)

df_4 = pd.read_csv("Deutschland_Cities_4.csv")
df_4.to_csv('C:\My projects\Streaming_Data_Projects\SparkStructuredStreaming_Project1\InputStreamingData\InputStreamingData\Deutschland_Cities_4.csv', 
            sep=',',index=False)

df_5 = pd.read_csv("Deutschland_Cities_5.csv")
df_5.to_csv('C:\My projects\Streaming_Data_Projects\SparkStructuredStreaming_Project1\InputStreamingData\InputStreamingData\Deutschland_Cities_5.csv', 
            sep=',',index=False)

df_6 = pd.read_csv("Deutschland_Cities_6.csv")
df_6.to_csv('C:\My projects\Streaming_Data_Projects\SparkStructuredStreaming_Project1\InputStreamingData\InputStreamingData\Deutschland_Cities_6.csv', 
            sep=',',index=False)

df_7 = pd.read_csv("Deutschland_Cities_7.csv")
df_7.to_csv('C:\My projects\Streaming_Data_Projects\SparkStructuredStreaming_Project1\InputStreamingData\InputStreamingData\Deutschland_Cities_7.csv', 
            sep=',',index=False)

In [5]:
df_1

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Berlin,52.5167,13.3833,Germany,DE,Berlin,primary,3644826.0,3644826.0
1,Hamburg,53.5500,10.0000,Germany,DE,Hamburg,admin,1841179.0,1841179.0
2,Munich,48.1372,11.5755,Germany,DE,Bavaria,admin,1471508.0,1471508.0
3,Cologne,50.9422,6.9578,Germany,DE,North Rhine-Westphalia,NaN,1085664.0,1085664.0
4,Frankfurt,50.1136,8.6797,Germany,DE,Hesse,minor,753056.0,753056.0
...,...,...,...,...,...,...,...,...,...
95,Villingen-Schwenningen,48.0603,8.4586,Germany,DE,Baden-Württemberg,minor,85181.0,85181.0
96,Konstanz,47.6633,9.1753,Germany,DE,Baden-Württemberg,NaN,84760.0,84760.0
97,Marl,51.6667,7.1167,Germany,DE,North Rhine-Westphalia,NaN,83941.0,83941.0
98,Worms,49.6319,8.3653,Germany,DE,Rhineland-Palatinate,minor,83330.0,83330.0


### Creating local SparkSession for Structured Streaming

In [6]:
def CreateSparkSession():
    try: 
        print("\033[92m SparkSession was successfully created")
        spark = SparkSession.builder.appName("StructuredStreamingReadFromCsvfiles").master('local[4]').getOrCreate() 
    except Exception:
        print("\033[91m SparkSession wasn't successfully created")
    return spark
    
spark = CreateSparkSession()

 SparkSession was successfully created


In [7]:
spark

### Creating streaming DataFrames and streaming Datasets
####  Reading all csv files written atomically in a directory


In [8]:
def CreateDataFrameForStreamingDataSets():
    try:
        schema1 = StructType([StructField('city',StringType(), True),
                        StructField('lat',DoubleType(), True),
                        StructField('lng',DoubleType(), True),
                        StructField('country',StringType(), True),
                        StructField('iso2',StringType(), True),
                        StructField('admin_name',StringType(), True),
                        StructField('capital',StringType(), True),
                        StructField('population',DoubleType(), True),
                        StructField('population_proper',DoubleType(), True),])
        LoadingPath = r"C:\My projects\Streaming_Data_Projects\SparkStructuredStreaming_Project1\InputStreamingData"
        Deustchland = spark.readStream.format("csv").schema(schema1)\
                        .option("header",True).option("maxFilesPerTrigger", 1)\
                        .load(LoadingPath)
        print("\033[92m DataFrame For Streaming Dataset was successfully created")
    except Exception:
        print("\033[91m DataFrame For Streaming Dataset wasn't successfully created")
    return Deustchland 

dfDeutschland = CreateDataFrameForStreamingDataSets()

 DataFrame For Streaming Dataset was successfully created


In [9]:
def StreamingDataFrame():
    try:
        dfDeutschland.printSchema()
        print("\033[92m The schema of the Streaming DataFrame was successfully created")
    except Exception:
        print("\033[91m The schema of the Streaming DataFrame wasn't successfully created")

StreamingDataFrame()

root
 |-- city: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- country: string (nullable = true)
 |-- iso2: string (nullable = true)
 |-- admin_name: string (nullable = true)
 |-- capital: string (nullable = true)
 |-- population: double (nullable = true)
 |-- population_proper: double (nullable = true)

 The schema of the Streaming DataFrame was successfully created


### Verifying if this dataframe has streaming sources

In [10]:
dfDeutschland.isStreaming

True

### Writing streaming data (DataFrame with no aggregations) to parquet files.

In [11]:
def WritingStreamingData():
    path = r"C:\My commercial projects\Data Engineering Projects\SparkStructuredStreaming_Project1\OutputNoAgg"
    return  dfDeutschland.writeStream.format("parquet")\
        .option("checkpointLocation",path)\
        .option("path",path)\
        .trigger(processingTime="1 second").start()
      
NoAggdfDeutschland = WritingStreamingData()

In [12]:
for x in range(10):
    print('\033[92m Status of the query')
    display(NoAggdfDeutschland.status)
    time.sleep(1)

 Status of the query


{'message': 'Initializing sources',
 'isDataAvailable': False,
 'isTriggerActive': False}

 Status of the query


{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

 Status of the query


...

{'message': 'Getting offsets from FileStreamSource[file:/C:/My commercial projects/Data Engineering Projects/SparkStructuredStreaming_Project1/InputStreamingData]',
 'isDataAvailable': True,
 'isTriggerActive': True}

 Status of the query


{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

 Status of the query


{'message': 'Writing offsets to log',
 'isDataAvailable': True,
 'isTriggerActive': True}

### Creating DataFrame with aggregation
#### Starting running the query and the output is stored in memory as an in-memory table
##### Putting all the aggregates in an in-memory table:

In [13]:
def AggDataFrame():
    dfDeutschlandAgg = dfDeutschland.groupBy("admin_name").sum("population")
    return dfDeutschlandAgg.writeStream \
    .queryName("GroupingDeutschlandCityStreaming") \
    .outputMode("complete").format("memory").start()

AggdfDeustchland = AggDataFrame()

### Veryfing of active streaming queries

In [14]:
spark.streams.active[0].isActive

True

### Displaying an in-memory table to look at streaming data

In [15]:
def InMemoryTable():
    for x in range(15):
        print('\033[93m Status of the query')
        display(AggdfDeustchland.status)
        print('\033[95m Watching continuously updated results')
        df = spark.sql("select * from GroupingDeutschlandCityStreaming")
        if df.count() > 0:
            df.show()
        time.sleep(1)

InMemoryTable()

 Status of the query


{'message': 'Getting offsets from FileStreamSource[file:/C:/My commercial projects/Data Engineering Projects/SparkStructuredStreaming_Project1/InputStreamingData]',
 'isDataAvailable': False,
 'isTriggerActive': True}

 Watching continuously updated results
 Status of the query


{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

 Watching continuously updated results
 Status of the query


...

 Watching continuously updated results
+--------------------+---------------+
|          admin_name|sum(population)|
+--------------------+---------------+
|Mecklenburg-Weste...|       577421.0|
|       Saxony-Anhalt|      1060149.0|
|         Brandenburg|       952848.0|
|              Berlin|      3644826.0|
|             Bavaria|      5167465.0|
|Rhineland-Palatinate|      1452355.0|
|              Saxony|      1992738.0|
|North Rhine-Westp...|    1.4580071E7|
|           Thuringia|      1029568.0|
|        Lower Saxony|      3650983.0|
|             Hamburg|      1841179.0|
|              Bremen|       838543.0|
|               Hesse|      2881998.0|
|   Baden-Württemberg|      4981429.0|
|            Saarland|       434268.0|
|  Schleswig-Holstein|      1144953.0|
+--------------------+---------------+

 Status of the query


{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

 Watching continuously updated results
+--------------------+---------------+
|          admin_name|sum(population)|
+--------------------+---------------+
|Mecklenburg-Weste...|       577421.0|
|       Saxony-Anhalt|      1060149.0|
|         Brandenburg|       952848.0|
|              Berlin|      3644826.0|
|             Bavaria|      5167465.0|
|Rhineland-Palatinate|      1452355.0|
|              Saxony|      1992738.0|
|North Rhine-Westp...|    1.4580071E7|
|           Thuringia|      1029568.0|
|        Lower Saxony|      3650983.0|
|             Hamburg|      1841179.0|
|              Bremen|       838543.0|
|               Hesse|      2881998.0|
|   Baden-Württemberg|      4981429.0|
|            Saarland|       434268.0|
|  Schleswig-Holstein|      1144953.0|
+--------------------+---------------+

 Status of the query


{'message': 'Getting offsets from FileStreamSource[file:/C:/My commercial projects/Data Engineering Projects/SparkStructuredStreaming_Project1/InputStreamingData]',
 'isDataAvailable': False,
 'isTriggerActive': True}

 Watching continuously updated results
+--------------------+---------------+
|          admin_name|sum(population)|
+--------------------+---------------+
|Mecklenburg-Weste...|       577421.0|
|       Saxony-Anhalt|      1060149.0|
|         Brandenburg|       952848.0|
|              Berlin|      3644826.0|
|             Bavaria|      5167465.0|
|Rhineland-Palatinate|      1463072.0|
|              Saxony|      1992738.0|
|North Rhine-Westp...|    1.4580071E7|
|           Thuringia|      1029568.0|
|        Lower Saxony|      3650983.0|
|             Hamburg|      1841179.0|
|              Bremen|       838543.0|
|               Hesse|      2881998.0|
|   Baden-Württemberg|      4981429.0|
|            Saarland|       434268.0|
|  Schleswig-Holstein|      1144953.0|
+--------------------+---------------+



### The final result of output sink:

In [16]:
def ResultOfOutput():
    spark.sql("select * from GroupingDeutschlandCityStreaming").show(truncate=False) 

ResultOfOutput()

+-----------------------------+---------------+
|admin_name                   |sum(population)|
+-----------------------------+---------------+
|Mecklenburg-Western Pomerania|577421.0       |
|Saxony-Anhalt                |1060149.0      |
|Brandenburg                  |952848.0       |
|Berlin                       |3644826.0      |
|Bavaria                      |5167465.0      |
|Rhineland-Palatinate         |1463072.0      |
|Saxony                       |1992738.0      |
|North Rhine-Westphalia       |1.4580071E7    |
|Thuringia                    |1029568.0      |
|Lower Saxony                 |3650983.0      |
|Hamburg                      |1841179.0      |
|Bremen                       |838543.0       |
|Hesse                        |2881998.0      |
|Baden-Württemberg            |4981429.0      |
|Saarland                     |434268.0       |
|Schleswig-Holstein           |1144953.0      |
+-----------------------------+---------------+



### Displaying an in-memory table

In [17]:
def DisplayInMemoryTable():
    spark.sql('show tables').show(truncate=False)

DisplayInMemoryTable()

+---------+--------------------------------+-----------+
|namespace|tableName                       |isTemporary|
+---------+--------------------------------+-----------+
|         |groupingdeutschlandcitystreaming|true       |
+---------+--------------------------------+-----------+



### Creating DataFrame based on an in-memory table and save this data to parquet and json files

In [18]:
def CreateDataFrameBasedOnInMemoryTable():
    sqlContext = SQLContext(spark)
    df=sqlContext.sql("select * from GroupingDeutschlandCityStreaming")
    return df 

df = CreateDataFrameBasedOnInMemoryTable()
df.show(truncate=False)

+-----------------------------+---------------+
|admin_name                   |sum(population)|
+-----------------------------+---------------+
|Mecklenburg-Western Pomerania|577421.0       |
|Saxony-Anhalt                |1060149.0      |
|Brandenburg                  |952848.0       |
|Berlin                       |3644826.0      |
|Bavaria                      |5167465.0      |
|Rhineland-Palatinate         |1463072.0      |
|Saxony                       |1992738.0      |
|North Rhine-Westphalia       |1.4580071E7    |
|Thuringia                    |1029568.0      |
|Lower Saxony                 |3650983.0      |
|Hamburg                      |1841179.0      |
|Bremen                       |838543.0       |
|Hesse                        |2881998.0      |
|Baden-Württemberg            |4981429.0      |
|Saarland                     |434268.0       |
|Schleswig-Holstein           |1144953.0      |
+-----------------------------+---------------+



c:\Users\dlugo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [19]:
def SavingToJsonParquetFiles():
    try:
        df.coalesce(1).write.mode("overwrite").parquet("GroupingDeutschlandCityStreaming.parquet")
        df.coalesce(1).write.mode("overwrite").json("GroupingDeutschlandCityStreaming.json")
        print("\033[92m DataFrame was successfully saved to JSON and parquet files")
    except Exception:
        print("\033[91m DataFrame wasn't successfully saved to JSON and parquet files")

SavingToJsonParquetFiles()

 DataFrame was successfully saved to JSON and parquet files


In [20]:
spark.streams.active[0].isActive

True

### Managing Streaming Queries

In [21]:
def ManagingStreamingQueries():
    print("\033[94m Managing Streaming Queries")
    recentProgress = AggdfDeustchland.recentProgress,
    lastProgress = AggdfDeustchland.lastProgress
    name = AggdfDeustchland.name
    explain = AggdfDeustchland.explain
    statusAggdf = AggdfDeustchland.status
    statusNoAggdf = NoAggdfDeutschland.status
    return recentProgress, lastProgress, name, explain, statusAggdf,statusNoAggdf

ManagingStreamingQueries()

 Managing Streaming Queries


(([{'id': '1dd2f6ca-8900-434c-8b04-e4edb656da4f',
    'runId': 'bbf01bdb-9af7-4d4f-9033-8747473d9e78',
    'name': 'GroupingDeutschlandCityStreaming',
    'timestamp': '2023-10-13T07:29:59.214Z',
    'batchId': 0,
    'numInputRows': 100,
    'inputRowsPerSecond': 0.0,
    'processedRowsPerSecond': 4.837695321948623,
    'durationMs': {'addBatch': 19500,
     'commitOffsets': 339,
     'getBatch': 0,
     'latestOffset': 384,
     'queryPlanning': 82,
     'triggerExecution': 20671,
     'walCommit': 366},
    'stateOperators': [{'operatorName': 'stateStoreSave',
      'numRowsTotal': 16,
      'numRowsUpdated': 16,
      'allUpdatesTimeMs': 715,
      'numRowsRemoved': 0,
      'allRemovalsTimeMs': 0,
      'commitTimeMs': 67834,
      'memoryUsedBytes': 49144,
      'numRowsDroppedByWatermark': 0,
      'numShufflePartitions': 200,
      'numStateStoreInstances': 200,
      'customMetrics': {'loadedMapCacheHitCount': 0,
       'loadedMapCacheMissCount': 0,
       'stateOnCurrentVersi

### `Stopping` queries:

In [22]:
def StopQueries():
    print("\033[91m Stopping queries")
    stopAggdf = AggdfDeustchland.stop()
    statusAggdf = AggdfDeustchland.status
    stopNoAggdf = NoAggdfDeutschland.stop()
    statusNoAggdf = NoAggdfDeutschland.status
    return stopAggdf, statusAggdf, stopNoAggdf, statusNoAggdf
    
StopQueries()

 Stopping queries


(None,
 {'message': 'Stopped', 'isDataAvailable': False, 'isTriggerActive': False},
 None,
 {'message': 'Stopped', 'isDataAvailable': False, 'isTriggerActive': False})